#### Test mynds packages in the sandbox

In [2]:
from collections.abc import Mapping
from functools import partial
from pathlib import Path
from typing import Callable, TypeVar

import tqdm

from mynd.image import ImageType, ImageComposite, ImageCompositeLoader
from mynd.io import read_image

from mynd.utils.filesystem import (
    list_directory,
    create_resource,
    Resource,
    ResourceManager,
)
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


IMAGE_PATTERN: str = "*.tiff"

Resources = list[Resource]


def collect_image_resources(directories: dict, tags: dict) -> ResourceManager:
    """Collects image resources and adds them to a manager."""
    image_manager: ResourceManager = ResourceManager()

    # Find file handles
    image_files: dict[ImageType, list[Path]] = {
        image_type: list_directory(directory, IMAGE_PATTERN)
        for image_type, directory in directories.items()
    }

    # Create resources out of the file handles
    for image_type, files in image_files.items():
        resources: Resources = [
            create_resource(path) for path in files if path.exists()
        ]

        if len(resources) == 0:
            logger.warning(f"no resources for type: {image_type}")

        image_manager.add_resources(resources, tags=tags.get(image_type))

    return image_manager


T: TypeVar = TypeVar("T")

ComponentGroups = Mapping[T, Resources]
ComponentMatch = Mapping[T, Resource]


def match_resources_by_stem(
    component_groups: ComponentGroups,
) -> list[ComponentMatch]:
    """Matches resources of different types based on common stems."""
    matches: dict[str, ComponentMatch] = dict()
    for key, resources in component_groups.items():

        for resource in resources:
            if resource.stem not in matches:
                matches[resource.stem] = dict()

            matches[resource.stem][key] = resource

    matches: list[ComponentMatch] = list(matches.values())
    return matches


def check_match_has_all_components(
    components: ComponentMatch,
    required_keys: set[T],
) -> bool:
    """Check if image matches has the required components."""
    return set(components.keys()) == required_keys


def create_resource_matcher(
    matcher: Callable[
        [ComponentGroups], dict[str, ComponentMatch]
    ] = match_resources_by_stem,
    validator: Callable[[ComponentMatch], bool] = lambda x: True,
) -> Callable:
    """Creates a resource matcher."""

    def match_resources(
        component_groups: ComponentGroups,
    ) -> list[ComponentMatch]:
        """Matches resources from different groups."""
        component_matches: list[ComponentMatch] = matcher(component_groups)
        valid_matches: list[ComponentMatch] = [
            components
            for components in component_matches
            if validator(components)
        ]
        return valid_matches

    return match_resources


def create_composite_loader(
    label: str, resources: Mapping[ImageType, Resource]
) -> ImageCompositeLoader:
    """Creates a composite loader"""

    def load_image_composite() -> tuple[str, ImageComposite]:
        """Loads an image composite from resources."""
        components: dict[ImageType, Image] = {
            image_type: read_image(resource.handle).unwrap()
            for image_type, resource in resources.items()
        }
        return label, ImageComposite(components)

    return load_image_composite


def create_image_composite_builder(
    loader_factory: Callable[
        [str, ComponentMatch], ImageCompositeLoader
    ] = create_composite_loader,
) -> Callable:
    """Creates a builder method for image composite loaders."""

    def build_image_composite_loaders(
        component_matches: list[Mapping[ImageType, Resource]],
    ) -> list[ImageCompositeLoader]:
        """Builds image composite loaders from a collection of image components."""

        labelled_composites: dict[str, Mapping] = dict()
        for components in component_matches:
            if ImageType.COLOR in components:
                label: str = components.get(ImageType.COLOR).stem
            else:
                label: str = [components.values()][0].stem

            labelled_composites[label] = components

        loaders: list[ImageCompositeLoader] = [
            loader_factory(label, components)
            for label, components in labelled_composites.items()
        ]

        return loaders

    return build_image_composite_loaders


def log_image_queries(
    query_results: Mapping[ImageType, list[Resource]]
) -> None:
    """Log some image query statistics."""
    for category, resources in query_results.items():
        file_count: int = len(
            [resource for resource in resources if resource.is_file()]
        )
        dir_count: int = len(
            [resource for resource in resources if resource.is_directory()]
        )

        logger.info(f"Category: {category}, resources: {len(resources)}")
        logger.info(f" - Files:        {file_count}")
        logger.info(f" - Directories:  {dir_count}")


def main() -> None:
    """Main entrypoint."""

    DATADIR: Path = Path("/home/martin/data")

    directories: dict[str, Path] = {
        ImageType.COLOR: DATADIR
        / Path("acfr_images_debayered/r23685bc_20100605_021022_debayered"),
        ImageType.RANGE: DATADIR
        / Path("acfr_stereo_ranges/r23685bc_20100605_021022_ranges"),
        ImageType.NORMAL: DATADIR
        / Path("acfr_stereo_normals/r23685bc_20100605_021022_normals"),
    }

    tags: dict[ImageType, list[str]] = {
        ImageType.COLOR: ["image", "color"],
        ImageType.RANGE: ["image", "range"],
        ImageType.NORMAL: ["image", "normal"],
    }

    image_types: list[ImageType] = list(directories.keys())

    # Step 1 - Collect image resources
    image_manager: ResourceManager = collect_image_resources(directories, tags)

    # Step 2 - Retrieve image resources by type
    query_results: dict[ImageType, list[Resource]] = {
        image_type: image_manager.query_tags(tags.get(image_type))
        for image_type in image_types
    }

    log_image_queries(query_results)

    # Step 3 - Match image resources to create composites
    matcher = create_resource_matcher()
    matches: list[ComponentMatch] = matcher(query_results)

    # Step 4 - Create loaders from the matched resources
    builder = create_image_composite_builder()
    loaders: list[tuple[str, ImageCompositeLoader]] = builder(matches)

    logger.info(f"loaders: {type(loaders)}")

    for loader in tqdm.tqdm(loaders, desc="loading images..."):
        label, composite = loader()
        # composite: ImageComposite = loader()


# INVOKE MAIN
main()

2024-10-09 22:25:17.233 | INFO     | __main__:log_image_queries:147 - Category: color, resources: 5458
2024-10-09 22:25:17.233 | INFO     | __main__:log_image_queries:148 -  - Files:        5458
2024-10-09 22:25:17.234 | INFO     | __main__:log_image_queries:149 -  - Directories:  0
2024-10-09 22:25:17.257 | INFO     | __main__:log_image_queries:147 - Category: range, resources: 5456
2024-10-09 22:25:17.258 | INFO     | __main__:log_image_queries:148 -  - Files:        5456
2024-10-09 22:25:17.258 | INFO     | __main__:log_image_queries:149 -  - Directories:  0
2024-10-09 22:25:17.282 | INFO     | __main__:log_image_queries:147 - Category: normal, resources: 5456
2024-10-09 22:25:17.283 | INFO     | __main__:log_image_queries:148 -  - Files:        5456
2024-10-09 22:25:17.284 | INFO     | __main__:log_image_queries:149 -  - Directories:  0
2024-10-09 22:25:17.441 | INFO     | __main__:main:193 - loaders: <class 'list'>
loading images...: 100%|██████████████████████████████████████████